In [1]:
from datetime import datetime as dt
import git
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import collections
import json
import time
from vehicle_stream_pipeline import utils 

In [2]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")

df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="MoDstops"
)
df_edges = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
)
df_edges.rename(columns={"Start #": "start_id", "Ende #": "end_id"}, inplace=True)

rides_df = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")
rides_df = rides_df[(rides_df["state"] == "completed")]
rides_df["scheduled_to"] = pd.to_datetime(rides_df["scheduled_to"])

/var/folders/kw/t27bnrpn52lfl7x3g64tdw1r0000gn/T/ipykernel_11523/666691188.py:11: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  rides_df = pd.read_csv(f"{repo}/data/cleaning/data_cleaned.csv")


In [3]:
pickup_address = utils.find_id_for_name('Mandelgasse', df_stops)
dropoff_address = utils.find_id_for_name('Schütt', df_stops)

In [4]:
radius = 500

In [5]:
start_date = rides_df.scheduled_to.min()
start_date = dt.strftime(start_date, "%Y-%m-%d")
end_date = rides_df.scheduled_to.max()
end_date = dt.strftime(end_date, "%Y-%m-%d")
start_date = dt.strptime(start_date, "%Y-%m-%d")
end_date = dt.strptime(end_date, "%Y-%m-%d")

In [22]:
start_month = start_date.month
start_year = start_date.year
end_month = end_date.month
end_year = end_date.year

In [23]:
print(start_month, start_year, end_month, end_year)

7 2021 5 2022


In [24]:
years = list(range(start_year, end_year + 1))
i = len(years)
j = 0
months = []
if i == 1: 
    months.extend(range(start_month, end_month + 1))
else: 
    while i > 1:
        if j == 0:
            months.extend(range(start_month, 13))
            j = j+1
        else: 
            months.extend(range(1, 13))
        i = i-1
    else: 
        months.extend(range(1, end_month +1))
        i = i-1

In [29]:
rides_df_1 = rides_df.copy()
sim_rides = 0
if sim_rides != 0:

        new_rides_all = pd.DataFrame(columns=rides_df_1.columns)
        for year in years:
            for month in months:
                new_rides = utils.generateRideSpecs(
                    rides_df_1,
                    new_rides_all,
                    df_stops,
                    df_edges,
                    sim_rides,
                    month,
                    year,
                )
                new_rides_all = pd.concat([new_rides, new_rides_all])

        new_rides_all["simulated"] = True
        rides_df_1["simulated"] = False

        new_rides_all = pd.concat([rides_df_1, new_rides_all])

else:
    new_rides_all = rides_df_1

In [32]:
rides_df_filterd = new_rides_all[
    (new_rides_all["scheduled_to"] > start_date)
    & (new_rides_all["scheduled_to"] < end_date)
]

In [34]:
drives_without_drones = utils.calculate_drives(
    rides_df_filterd, start_date, end_date
)

In [35]:
hotspots = [1008, 4025, 1005, 1009, 1007, 12007, 7001, 6004, 1010, 11017]
drone_spots = [15011, 13001, 2002, 11007, 4016, 1002, 3020, 9019, 9005]
df_stops_drones = df_stops[df_stops["MoDStop Id"].isin(drone_spots)]

In [43]:
# if drones not activated
layers = []

graph_without_drones = utils.calculate_graph(drives_without_drones)
path, shortest_time = utils.get_shortest_ride(
    pickup_address, dropoff_address, graph_without_drones
)
route_information = utils.get_route_information(
    drives_without_drones, path, df_stops
)

In [45]:
print(path, route_information)

[1001, 1005, 4025, 1009] The shortest path is :
 
Mandelgasse - Rathaus: 17.63312 days
 
Rathaus - Rathaus Hambach: 17.63945 days
 
Rathaus Hambach - Schütt: 7.98268 days


In [ ]:
# if drones activated 
layers = utils.create_circles_around_drone_spots(df_stops_drones, radius)

drives_with_drones = utils.add_drone_flights(
    df_edges, drives_without_drones, drone_spots=drone_spots, radius=radius
)

graph_with_drones = utils.calculate_graph(drives_with_drones)

path_w_d, shortest_time = utils.get_shortest_ride(
    pickup_address, dropoff_address, graph_with_drones
)

route_information_w_d = utils.get_route_information(
    drives_with_drones, path_w_d, df_stops
)

In [ ]:
print(path_w_d, route_information_w_d)

In [ ]:
pickup_counts = (
    rides_df_filterd.groupby("pickup_address")
    .size()
    .to_frame("number_of_pickups")
    .reset_index()
)
dropoff_counts = (
    rides_df_filterd.groupby("dropoff_address")
    .size()
    .to_frame("number_of_dropoffs")
    .reset_index()
)
pickup_counts["pickup_address"] = pickup_counts["pickup_address"].astype(int)
dropoff_counts["dropoff_address"] = dropoff_counts["dropoff_address"].astype(int)

In [ ]:
df_stops_1 = pd.merge(
    df_stops, pickup_counts, left_on="MoDStop Id", right_on="pickup_address"
).drop("pickup_address", axis=1)

In [ ]:
df_stops_1 = pd.merge(
    df_stops_1, dropoff_counts, left_on="MoDStop Id", right_on="dropoff_address"
).drop("dropoff_address", axis=1)